# HPO介绍

![hello](./HPO.png)

# 贝叶斯优化

在贝叶斯优化的数学过程当中，我们主要执行以下几个步骤：

1 定义需要估计的f(x)以及x的定义域

2 取出有限的n个x上的值，求解出这些x对应的f(x)（求解观测值）

3 根据有限的观测值，对函数进行估计（该假设被称为贝叶斯优化中的先验知识），得出该估计$f^∗$上的目标值（最大值或最小值）

4 定义某种规则，以确定下一个需要计算的观测点

并持续在2-4步骤中进行循环，直到假设分布上的目标值达到我们的标准，或者所有计算资源被用完为止（例如，最多观测m次，或最多允许运行t分钟）。

以上流程又被称为序贯模型优化（SMBO），是最为经典的贝叶斯优化方法。在实际的运算过程当中，尤其是超参数优化的过程当中，有以下具体细节需要注意：

当贝叶斯优化不被用于HPO时，一般f(x)可以是完全的黑盒函数（black box function，也译作黑箱函数，即只知道x与f(x)的对应关系，却丝毫不知道函数内部规律、同时也不能写出具体表达式的一类函数），因此贝叶斯优化也被认为是可以作用于黑盒函数估计的一类经典方法。但在HPO过程当中，需要定义的f(x)一般是交叉验证的结果/损失函数的结果，而我们往往非常清楚损失函数的表达式，只是我们不了解损失函数内部的具体规律，因此HPO中的f(x)不能算是严格意义上的黑盒函数。

在HPO中，自变量x就是超参数空间。在上述二维图像表示中，x为一维的，但在实际进行优化时，超参数空间往往是高维且极度复杂的空间。

最初的观测值数量n、以及最终可以取到的最大观测数量m都是贝叶斯优化的超参数，最大观测数量m也决定了整个贝叶斯优化的迭代次数。

在第3步中，根据有限的观测值、对函数分布进行估计的工具被称为概率代理模型（Probability Surrogate model），毕竟在数学计算中我们并不能真的邀请数万人对我们的观测点进行连线。这些概率代理模型自带某些假设，他们可以根据廖廖数个观测点估计出目标函数的分布$f^∗$（包括$f^∗$上每个点的取值以及该点对应的置信度）。在实际使用时，概率代理模型往往是一些强大的算法，最常见的比如高斯过程、高斯混合模型等等。传统数学推导中往往使用高斯过程，但现在最普及的优化库中基本都默认使用基于高斯混合模型的TPE过程。

在第4步中用来确定下一个观测点的规则被称为采集函数（Aquisition Function），采集函数衡量观测点对拟合$f^∗$所产生的影响，并选取影响最大的点执行下一步观测，因此我们往往关注采集函数值最大的点。最常见的采集函数主要是概率增量PI（Probability of improvement，比如我们计算的频数）、期望增量（Expectation Improvement）、置信度上界（Upper Confidence Bound）、信息熵（Entropy）等等。上方gif图像当中展示了PI、UCB以及EI。其中大部分优化库中默认使用期望增量。

在HPO中使用贝叶斯优化时，我们常常会看见下面的图像，这张图像表现了贝叶斯优化的全部基本元素，我们的目标就是在采集函数指导下，让$f^∗$尽量接近f(x)。

![hello](./贝叶斯优化.png)

# bayes_opt

In [2]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import numpy as np

# 产生随机分类数据集，10个特征， 2个类别
x, y = make_classification(n_samples=1000,n_features=10,n_classes=2)

In [3]:
rf = RandomForestClassifier()
print(np.mean(cross_val_score(rf, x, y, cv=20, scoring='roc_auc')))

0.9835200000000001


In [4]:
def rf_cv(n_estimators, min_samples_split, max_features, max_depth):
    val = cross_val_score(
        RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=2
        ),
        x, y, scoring='roc_auc', cv=5
    ).mean()
    return val

In [5]:
rf_bo = BayesianOptimization(
    rf_cv, {
        'n_estimators': (10, 250),
        'min_samples_split': (2, 25),
        'max_features': (0.1, 0.999),
        'max_depth': (5, 15)
    })

In [6]:
rf_bo.maximize()

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.9834    | 13.36     | 0.3301    | 23.91     | 105.0     |
| 2         | 0.9808    | 7.901     | 0.8909    | 22.53     | 245.8     |
| 3         | 0.9808    | 13.3      | 0.1175    | 13.65     | 183.9     |
| 4         | 0.9822    | 11.84     | 0.7042    | 24.64     | 156.0     |
| 5         | 0.9831    | 9.784     | 0.5724    | 9.969     | 218.7     |
| 6         | 0.9816    | 14.07     | 0.7382    | 24.9      | 104.3     |
| 7         | 0.9814    | 5.214     | 0.2656    | 21.84     | 230.2     |
| 8         | 0.9835    | 5.048     | 0.3346    | 3.791     | 225.8     |
| 9         | 0.9827    | 13.99     | 0.4269    | 20.41     | 220.7     |
| 10        | 0.9829    | 9.444     | 0.5339    | 21.15     | 188.3     |
| 11        | 0.9828    | 14.77     | 0.4967    | 11.82     | 209.6     |
| 12        | 0.9806    | 8.171     | 

In [7]:
rf_bo.max

{'target': 0.983459809980998,
 'params': {'max_depth': 5.048341014337014,
  'max_features': 0.33457277561746435,
  'min_samples_split': 3.7906637588173613,
  'n_estimators': 225.80156694493454}}

# hyperopt

In [15]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import matplotlib.pyplot as plt
import numpy as np, pandas as pd
from math import *
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier as kNN
from sklearn.model_selection import cross_val_score

# 数据集导入
iris = datasets.load_iris()
X = iris.data
y = iris.target

# 损失函数
def hyperopt_train_test(params):
    clf = kNN(**params)
    return cross_val_score(clf, X, y).mean()

# hp.choice(label, options) 其中options应是 python 列表或元组
# space4nn就是需要输入到损失函数里面的参数
space4knn = {
    'n_neighbors': hp.choice('n_neighbors', range(1,100))
}

# 定义目标函数
def f(params):
    acc = hyperopt_train_test(params)
    return {'loss': -acc, 'status': STATUS_OK}

# Trials对象允许我们在每个时间步存储信息
trials = Trials()

# 函数fmin首先接受一个函数来最小化，algo参数指定搜索算法，最大评估次数max_evals
best = fmin(f, space4knn, algo=tpe.suggest, max_evals=100, trials=trials)
print('best:',best)
print('trials:')
for trial in trials.trials[:2]:
    print(trial)

100%|█████████████████████████████████████████████| 100/100 [00:04<00:00, 20.97trial/s, best loss: -0.9800000000000001]
best: {'n_neighbors': 11}
trials:
{'state': 2, 'tid': 0, 'spec': None, 'result': {'loss': -0.9400000000000001, 'status': 'ok'}, 'misc': {'tid': 0, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'n_neighbors': [0]}, 'vals': {'n_neighbors': [38]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2023, 10, 17, 10, 23, 22, 173000), 'refresh_time': datetime.datetime(2023, 10, 17, 10, 23, 22, 187000)}
{'state': 2, 'tid': 1, 'spec': None, 'result': {'loss': -0.9400000000000001, 'status': 'ok'}, 'misc': {'tid': 1, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'n_neighbors': [1]}, 'vals': {'n_neighbors': [44]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2023, 10, 17, 10, 23, 22, 191000), 'refresh_time': datetime.datetime(2023, 10, 17, 10, 23, 22, 203000)}


# optuna

In [8]:
import optuna
import numpy as np
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

In [9]:
# 糖尿病数据集
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()

X = diabetes.data  # data
y = diabetes.target  # label

In [10]:
def optuna_objective(trial):

    #定义参数空间
    n_estimators = trial.suggest_int("n_estimators", 80, 100,
                                     1)  #整数型，(参数名称，下界，上界，步长)
    max_depth = trial.suggest_int("max_depth", 10, 25, 1)
    max_features = trial.suggest_int("max_features", 10, 20, 1)
    #max_features = trial.suggest_categorical("max_features",["log2","sqrt","auto"]) #字符型
    min_impurity_decrease = trial.suggest_int("min_impurity_decrease", 0, 5, 1)
    #min_impurity_decrease = trial.suggest_float("min_impurity_decrease",0,5,log=False) #浮点型

    #定义评估器
    #需要优化的参数由上述参数空间决定
    #不需要优化的参数则直接填写具体值
    reg = RFR(n_estimators=n_estimators,
              max_depth=max_depth,
              max_features=max_features,
              min_impurity_decrease=min_impurity_decrease,
              random_state=1412,
              verbose=False,
              n_jobs=-1)

    #交叉验证过程，输出负均方根误差(-RMSE)
    #optuna同时支持最大化和最小化，因此如果输出-RMSE，则选择最大化
    #如果选择输出RMSE，则选择最小化
    cv = KFold(n_splits=5, shuffle=True, random_state=1412)
    validation_loss = cross_validate(
        reg,
        X,
        y,
        scoring="neg_root_mean_squared_error",
        cv=cv,  #交叉验证模式
        verbose=False,  #是否打印进程
        n_jobs=-1,  #线程数
        error_score='raise')
    #最终输出RMSE
    return np.mean(abs(validation_loss["test_score"]))

In [11]:
def optimizer_optuna(n_trials):

    algo = optuna.samplers.TPESampler(n_startup_trials=10, n_ei_candidates=24)

    #实际优化过程，首先实例化优化器
    study = optuna.create_study(
        sampler=algo,  #要使用的具体算法
        direction="minimize"  #优化的方向，可以填写minimize或maximize
    )
    #开始优化，n_trials为允许的最大迭代次数
    #由于参数空间已经在目标函数中定义好，因此不需要输入参数空间
    study.optimize(
        optuna_objective,  #目标函数
        n_trials=n_trials,  #最大迭代次数（包括最初的观测值的）
        show_progress_bar=True  #要不要展示进度条
    )

    #可直接从优化好的对象study中调用优化的结果
    #打印最佳参数与最佳损失值
    print("\n", "\n", "best params: ", study.best_trial.params, "\n", "\n",
          "best score: ", study.best_trial.values, "\n")

    return study.best_trial.params, study.best_trial.values

In [12]:
best_params, best_score = optimizer_optuna(10) #默认打印迭代过程

[I 2023-10-17 17:31:19,928] A new study created in memory with name: no-name-ed08082a-bfdf-425c-a8ef-9104d4e12212


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-10-17 17:31:25,306] Trial 0 finished with value: 57.67220867261342 and parameters: {'n_estimators': 100, 'max_depth': 19, 'max_features': 10, 'min_impurity_decrease': 1}. Best is trial 0 with value: 57.67220867261342.
[I 2023-10-17 17:31:27,176] Trial 1 finished with value: 57.509325481560055 and parameters: {'n_estimators': 89, 'max_depth': 17, 'max_features': 12, 'min_impurity_decrease': 5}. Best is trial 1 with value: 57.509325481560055.
[I 2023-10-17 17:31:29,077] Trial 2 finished with value: 57.60779671740814 and parameters: {'n_estimators': 90, 'max_depth': 21, 'max_features': 18, 'min_impurity_decrease': 0}. Best is trial 1 with value: 57.509325481560055.
[I 2023-10-17 17:31:30,374] Trial 3 finished with value: 57.66094761630933 and parameters: {'n_estimators': 98, 'max_depth': 12, 'max_features': 14, 'min_impurity_decrease': 1}. Best is trial 1 with value: 57.509325481560055.
[I 2023-10-17 17:31:30,735] Trial 4 finished with value: 57.642000518278245 and parameters: {'n